In [12]:
#Import all relevant libraries
import dtale
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import sweetviz as sv
import dtale.global_state as global_state
global_state.set_app_settings(dict(enable_custom_filters=True))
## This statement allows the visuals to render within your Jupyter Notebook.
%matplotlib inline

2024-05-14 03:39:45,072 - WARNING  - Turning on custom filtering. Custom filters are vulnerable to code injection attacks, please only use in trusted environments.


## Loading the data
We can now load the dataset into pandas using the read_csv() function. This converts the CSV file into a Pandas dataframe.

In [32]:
#Read in the csv file and convert to a Pandas dataframe
Jadarat_df = pd.read_csv("/Users/raneemaj/Documents/GitHub/Use-case-5/Jadarat_data.csv")
 
dtale.show(Jadarat_df, open_browser=True)

/opt/anaconda3/lib/python3.11/site-packages/dtale/views.py:816: FutureWarning:

['comp_size', 'eco_activity', 'qualif'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.

2024-05-14 04:56:01,317 - WARNING  - Custom filtering enabled. Custom filters are vulnerable to code injection attacks, please only use in trusted environments.


2024-05-14 05:00:55,585 - ERROR    - Exception occurred while processing request: multi-line expressions are only valid in the context of data, use DataFrame.eval
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/dtale/views.py", line 2080, in test_filter
    run_query(
  File "/opt/anaconda3/lib/python3.11/site-packages/dtale/query.py", line 158, in run_query
    df = df.query(
         ^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pandas/core/frame.py", line 4474, in query
    res = self.eval(expr, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pandas/u

### Viewing the dataframe
We can get a quick sense of the size of our dataset by using the shape method. This returns a tuple with the number of rows and columns in the dataset. 

## 1. Data Profiling:
Data profiling is a comprehensive process of examining the data available in an existing dataset and collecting statistics and information about that data. 

In [9]:
Jadarat_df.head()

,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender
0,محاسب,27/05/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,شركة مقر العالم للسفريات,1-317262,خاص,متوسطة فئة أ,أنشطة وكالات السياحة والسفر,"['Language data', 'اللغة الانجليزية', 'محترف']",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026350419,0 Years,both
1,بائع,27/05/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",شركة عالم الكهرباء للمقاولات,4-1324428,خاص,متوسطة فئة ب,تركيب انظمة التبريد وتكييف الهواء وصيانتها واص...,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary', '5000.0']",دوام كامل,0 / 3,20202026350389,0 Years,both
2,أخصائي عمليات موارد بشرية,27/05/1444,['تنفيذ الإجراءات والأنظمة والنماذج الخاصة بمر...,[' تنفيذ الإجراءات والأنظمة والنماذج الخاصة ...,شركه دار السلام,1-155294,خاص,متوسطة فئة أ,ترميمات المباني السكنية والغير سكنية,"['Language data', 'الانجليزيه', 'محترف']",الرياض,ATH THUMA...,"['Salary', '4000.0']",دوام كامل,0 / 2,20202026350347,2 Years,both
3,ميكانيكي سيارات,27/05/1444,['تشخيص أعطال السيارات وإصلاحها وتنفيذ برامج ا...,[' فحص أداء المعدّات الكهربائية والميكانيكية...,مؤسسة لمكو لغيار الزيوت,8-1925495,خاص,صغيرة فئة ب,NaN,"['Skill data', 'صيانة السيارات وتقييم الاعطال'...",المنطقة الشرقية,AL HUFUF...,"['Salary', '5000.0']",دوام كامل,0 / 10,20202026350219,0 Years,M
4,محاسب,27/05/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,مؤسسة فكرة اليمامة للمقاولات,1-2356639,خاص,كبيرة,الإنشاءات العامة للمباني السكنية,"['Skill data', 'تحمل ضغط العمل', 'محترف', 'Lan...",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026350043,0 Years,both


In [7]:
Jadarat_df.tail(10)

,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender
1460,كيميائي,23/06/1444,['دراسة خصائص المواد الكميائية للصناعات الكيمي...,[' إعداد برامج تشغيل الأجهزة والمعدات المخبر...,شركة نواف صالح البطي للتجارة,1-2604723,خاص,صغيرة فئة ب,تجارة الجملة والتجزئه في أنواع أخرى من المركبا...,NaN,الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026402275,0 Years,F
1461,مساعد إداري,23/06/1444,['متابعة تنفيذ القرارات الادارية الصادرة عن ال...,[' تنفيذ التعليمات الادارية الصادرة عن الرئي...,شركة النصرة للمحاماة والاستشارات القانونية شرك...,15-1970732,خاص,صغيرة فئة أ,أنشطة الاستشارات في مجال الزكاة وضريبة الدخل,"['Skill data', 'إدارة برامج التواصل الاجتماعي'...",المنطقة الشرقية,AL KHUBAR...,"['Salary', '4000.0']",دوام كامل,0 / 1,20202026402243,2 Years,M
1462,سكرتير قانوني,23/06/1444,['توفير الدعم الإداري للعاملين في المهن القانو...,[' جمع المراسلات والوثائق الخاصة بملف القضاي...,شركة النصرة للمحاماة والاستشارات القانونية شرك...,15-1970732,خاص,صغيرة فئة أ,أنشطة الاستشارات في مجال الزكاة وضريبة الدخل,"['Skill data', 'إدارة برامج التواصل الاجتماعي'...",المنطقة الشرقية,AL KHUBAR...,"['Salary', '4000.0']",دوام كامل,0 / 1,20202026402227,0 Years,M
1463,بائع,21/06/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",مؤسسة مصلح دخيل الله صالح الثبيتي للتجارة,13-13828,خاص,صغيرة فئة ب,البيع بالتجزئة للزهور والورود,"['Skill data', 'استخدام الحاسب ', 'محترف', 'La...",مكة المكرمة,MAKKAH AL...,"['Salary', '4000.0']",دوام كامل,0 / 1,20202026401331,0 Years,M
1464,بائع,21/06/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",شركة النجمة الذهبية للتبريد والتكييف شركة شخص ...,8-1977608,خاص,صغيرة فئة ب,تركيب انظمة التبريد وتكييف الهواء وصيانتها واص...,NaN,المنطقة الشرقية,AL HUFUF...,"['Salary', '4000.0']",دوام كامل,0 / 1,20202026401107,2 Years,F
1465,مهندس تقنية معلومات,19/06/1444,['دراسة احتياجات العميل من الأجهزة الحاسوبية و...,[' متابعة وتقييم عملية جمع وتحليل المعطيات و...,مطعم الاوراق الخضر لتقديم الوجبات,15-1698448,خاص,صغيرة فئة ب,المطاعم مع الخدمة,NaN,الرياض,AR RIYADH...,"['Salary', '5500.0']",دوام كامل,0 / 1,20202026400353,0 Years,both
1466,سكرتير,19/06/1444,['دعم المدير أو الإدارة من خلال كتابة الوثائق ...,[' تدوين الملاحظات، التي يتم إملاؤها، بطريقة...,شركة ملتقى التجار لحاضنات و مسرعات الأعمال,9-2013930,خاص,صغيرة فئة ب,أنشطة حاضنات ومسرعات الأعمال,"['Skill data', 'فن التواصل', 'محترف', 'Languag...",مكة المكرمة,JEDDAH...,"['Salary', '4000.0']",دوام كامل,0 / 1,20202026399849,2 Years,both
1467,صيدلي,18/06/1444,['دراسة التركيبات الدوائية للعقاقير والعلاجات ...,[' دراسة مواصفات وتركيبات الادوية والعقاقير ...,شركة تقنية الدواجن والألبان للتجارة,1-2415250,خاص,صغيرة فئة ب,NaN,"['Professional certificate \\ License data', '...",الرياض,AR RIYADH...,"['Salary', '7000.0']",دوام كامل,0 / 1,20202026399061,0 Years,both
1468,مشغل آلة تشطيب رخام,18/06/1444,['تشغيل آلة تشطيب الرخام وابعادء الأجزاء المتف...,[' تسوية الرخام بواسطة أقراص الألماس الحديدي...,مصنع عبدالرحمن بن محمد بن عبدالعزيز العبدالقاد...,15-1971895,خاص,صغيرة فئة أ,NaN,"['Skill data', 'متقن للعمل وشغوف ويرغب بالتطور...",المنطقة الشرقية,AL KHUBAR...,"['Salary', '4000.0', 'Benefits', 'اجازة سنوية ...",دوام كامل,0 / 3,20202026398972,0 Years,M
1469,صيدلي,18/06/1444,['دراسة التركيبات الدوائية للعقاقير والعلاجات ...,[' دراسة مواصفات وتركيبات الادوية والعقاقير ...,شركة تقنية الدواجن والألبان للتجارة,1-2415250,خاص,صغيرة فئة ب,NaN,"['Professional certificate \\ License data', '...",الرياض,AR RIYADH...,"['Salary', '7000.0']",دوام كامل,0 / 1,20202026399061,0 Years,both


In [133]:
Jadarat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_title     1470 non-null   object
 1   job_date      1470 non-null   object
 2   job_desc      1470 non-null   object
 3   job_tasks     1470 non-null   object
 4   comp_name     1470 non-null   object
 5   comp_no       1470 non-null   object
 6   comp_type     1470 non-null   object
 7   comp_size     1456 non-null   object
 8   eco_activity  1380 non-null   object
 9   qualif        780 non-null    object
 10  region        1470 non-null   object
 11  city          1470 non-null   object
 12  benefits      1470 non-null   object
 13  contract      1470 non-null   object
 14  positions     1470 non-null   object
 15  job_post_id   1470 non-null   int64 
 16  exper         1470 non-null   object
 17  gender        1470 non-null   object
dtypes: int64(1), object(17)
memory usage: 206.8+ KB


In [ ]:
# the most missing values are in the qualification(qualif) column

In [134]:
Jadarat_df.describe()


,job_post_id
count,1.470000e+03
mean,2.020203e+13
std,5.863941e+04
min,2.020203e+13
25%,2.020203e+13
50%,2.020203e+13
75%,2.020203e+13
max,2.020203e+13


In [31]:
Jadarat_df.shape

(1470, 18)

### Data Quality Checks
Data quality checks involve the process of ensuring that the data is accurate, complete, consistent, relevant, and reliable. 


**Here are typical steps involved in checking data quality:**

#### 1. Reliability:
Evaluate the data's source and collection process to determine its trustworthiness.

In [103]:
# as mentioned in the pdf the data are from Jadarat website which is a trustworthy website

#### 2. Timeliness: 
Ensure the data is up-to-date and reflective of the current situation or the period of interest for the analysis.

In [104]:
# the last version of the data was updated before 9 months so i might consider as an up-to-date data

In [105]:
#Jadarat_df.sort_values(by="job_date",ascending=True)

#### 3. Consistency: 

Confirm that the data is consistent within the dataset and across multiple data sources. For example, the same data point should not have different values in different places.


In [ ]:
# the data is consistent based on my researches

In [136]:
Jadarat_df.dtypes

job_title       object
job_date        object
job_desc        object
job_tasks       object
comp_name       object
comp_no         object
comp_type       object
comp_size       object
eco_activity    object
qualif          object
region          object
city            object
benefits        object
contract        object
positions       object
job_post_id      int64
exper           object
gender          object
dtype: object

#### 4. Relevance: 



In [137]:
Jadarat_df.head(2)

,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender
0,محاسب,27/05/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,شركة مقر العالم للسفريات,1-317262,خاص,متوسطة فئة أ,أنشطة وكالات السياحة والسفر,"['Language data', 'اللغة الانجليزية', 'محترف']",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026350419,0 Years,both
1,بائع,27/05/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",شركة عالم الكهرباء للمقاولات,4-1324428,خاص,متوسطة فئة ب,تركيب انظمة التبريد وتكييف الهواء وصيانتها واص...,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary', '5000.0']",دوام كامل,0 / 3,20202026350389,0 Years,both


In [108]:
#All the data are relevance 

## 1. Data Profiling:
Data profiling is a comprehensive process of examining the data available in an existing dataset and collecting statistics and information about that data. 

#### 5. Uniqueness: 
Check for and remove duplicate records to prevent skewed analysis results.


In [36]:
Jadarat_df.duplicated().sum()

1

In [122]:
#duplicated_values = Jadarat_df[Jadarat_df.duplicated()]
#print(duplicated_values)

In [37]:
job_df=Jadarat_df.copy()

In [38]:
job_df.drop_duplicates(inplace=True)

In [39]:
job_df.duplicated().sum()

0

In [40]:
job_df.reset_index(drop=True, inplace=True)

#### 6. Completeness: 
Ensure that no critical data is missing. This might mean checking for null values or required fields that are empty.

We will start by checking the dataset for missing or null values. For this, we can use the isna() method which returns a dataframe of boolean values indicating if a field is null or not. To group all missing values by column, we can include the sum() method.

In [123]:
#Display number missing values per column
missing_values = Jadarat_df.isnull().sum()
print(missing_values)

job_title         0
job_date          0
job_desc          0
job_tasks         0
comp_name         0
comp_no           0
comp_type         0
comp_size        14
eco_activity     90
qualif          690
region            0
city              0
benefits          0
contract          0
positions         0
job_post_id       0
exper             0
gender            0
dtype: int64


In [115]:
job_df.shape

(1469, 18)

In [ ]:
# go to clean them 

#### 7. Check Accuracy:

Verify that the data is correct and precise. This could involve comparing data samples with known sources or using validation rules.

**The process includes:**
1. Validating the appropriateness of data types for the dataset.
2. Identifying outliers  using established validation  rule

In [126]:
# check columns types 

# already checked above 

In [ ]:
# go to clean them 

In [127]:
# check outliers 

In [ ]:
# go to lesson: statistics 1 then statistics 3
# then go to univariate Non graphical analysis
# detect outliers using numerical statistics 

In [ ]:
# go to delete ouliers

## 2. Data Cleaning: 

Preliminary findings from data profiling can lead to cleaning the data by:
- Handling missing values
- Correcting errors.
- Dealing with outliers.

-------------------



## Q1:What proportion of job postings is attributed to each region within the kingdom?

![alt text](<Screenshot 1445-11-06 at 1.32.04 AM.png>)

![alt text](<Screenshot 1445-11-06 at 1.01.18 AM.png>)

## Q2: Is there a gender preference indicated in the job postings?

### There's a slightly different and it might be because some jobs being more suitable or socially acceptable for men

![alt text](<Screenshot 1445-11-06 at 1.52.48 AM.png>)

## Q3: What is the expected salary range for fresh graduates?

In [50]:

job_df['numeric_benefits'] = job_df['benefits'].str.extract(r'(\d+\.\d+|\d+)', expand=False).astype(float)

# Generate Sweetviz report
report = sv.analyze(job_df)

# Display the report in the web browser
report.show_html()

                                             |          | [  0%]   00:00 -> (? left)

/opt/anaconda3/lib/python3.11/site-packages/sweetviz/graph.py:33: UserWarning:

Glyph 1588 (\N{ARABIC LETTER SHEEN}) missing from current font.

/opt/anaconda3/lib/python3.11/site-packages/sweetviz/graph.py:33: UserWarning:

Matplotlib currently does not support Arabic natively.

/opt/anaconda3/lib/python3.11/site-packages/sweetviz/graph.py:33: UserWarning:

Glyph 1576 (\N{ARABIC LETTER BEH}) missing from current font.

/opt/anaconda3/lib/python3.11/site-packages/sweetviz/graph.py:33: UserWarning:

Glyph 1607 (\N{ARABIC LETTER HEH}) missing from current font.

/opt/anaconda3/lib/python3.11/site-packages/sweetviz/graph.py:33: UserWarning:

Glyph 1581 (\N{ARABIC LETTER HAH}) missing from current font.

/opt/anaconda3/lib/python3.11/site-packages/sweetviz/graph.py:33: UserWarning:

Glyph 1603 (\N{ARABIC LETTER KAF}) missing from current font.

/opt/anaconda3/lib/python3.11/site-packages/sweetviz/graph.py:33: UserWarning:

Glyph 1608 (\N{ARABIC LETTER WAW}) missing from current font.

/opt

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### most of them in 5000 range 

![alt text](<Screenshot 1445-11-06 at 8.50.38 AM.png>)

## Q4: Are job opportunities predominantly targeted at individuals with experience, or is there room for fresh graduates as well?

### as u can see in the chart below, most of the opportunites focusing on people with 0 experience (Freshly graduate)

![alt text](<Screenshot 1445-11-06 at 3.21.50 AM.png>)